In [30]:
import numpy as np
import pandas as pd
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scipy import stats   # print multiple line code

In [2]:
pan_number = pd.read_excel('PAN Number Validation Dataset.xlsx')
pan_number

,Pan_Numbers
0,VGLOD3180G
1,PHOXD7232L
2,MGEPH6532A
3,JJCHK4574O
4,XTQIJ2330L
...,...
9995,TNGUY429!V
9996,SMMN33673g
9997,NaN
9998,RLAI69795t


In [17]:
pan_number.shape[0]   #total row

10000

In [3]:
Total_Records = (len(pan_number))
Total_Records

10000

In [4]:
# change data type
pan_number['Pan_Numbers'] = pan_number['Pan_Numbers'].astype(str).str.strip().str.upper()

pan_number.dtypes
pan_number.head()

Pan_Numbers    object
dtype: object

,Pan_Numbers
0,VGLOD3180G
1,PHOXD7232L
2,MGEPH6532A
3,JJCHK4574O
4,XTQIJ2330L


In [120]:
pan_number['Pan_Numbers'].isnull().sum()

np.int64(0)

In [27]:
pan_number['Pan_Numbers'] == ""

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Name: Pan_Numbers, Length: 10000, dtype: bool

In [121]:
pan_number[pan_number['Pan_Numbers'] == ""]
pan_number[pan_number['Pan_Numbers'] == ""].count()

,Pan_Numbers
5019,
5020,


Pan_Numbers    2
dtype: int64

In [122]:
pan_number[pan_number['Pan_Numbers'] ==  "NAN"].count()

Pan_Numbers    965
dtype: int64

In [5]:
#Replace missing value
pan_number = pan_number.replace({"Pan_Numbers":""},"NAN")
pan_number[pan_number == "NAN"].count()
pan_number = pan_number.replace({"Pan_Numbers":"NAN"},pd.NA)
pan_number.isna().sum()

Pan_Numbers    967
dtype: int64

Pan_Numbers    967
dtype: int64

In [13]:
# def find_empty_pan_values(df, column_name):
#     # Create comprehensive mask for all types of empty/missing values
#     mask = (
#         df[column_name].isna() |
#         (df[column_name].astype(str).str.strip() == "") |
#         (df[column_name].astype(str).str.upper() == "NAN") |
#         (df[column_name].astype(str).str.upper() == "NULL") |
#         (df[column_name].astype(str).str.upper() == "N/A")
#     )
    
#     empty_count = mask.sum()
#     print(f"Total empty/missing values in '{column_name}': {empty_count}")
    
#     # Show rows with empty values
#     if empty_count > 0:
#         print(f"\nRows with empty {column_name}:")
#         print(df[mask])
    
#     return mask

# # Use the function
# empty_mask = find_empty_pan_values(pan_number, 'Pan_Numbers')

In [6]:
# Drop Null/NA Values
pan_number = pan_number.dropna()
pan_number.shape[0]
non_missing_pan_number = pan_number.count()
print(non_missing_pan_number)

9033

Pan_Numbers    9033
dtype: int64


In [129]:
len(non_missing_pan_number)

9033

In [130]:
print("Unique PAN Number",pan_number['Pan_Numbers'].nunique())  #total unique PAN NUMBER


Unique PAN Number 9025


In [7]:
# Remove Duplicates
pan_number = pan_number.drop_duplicates(subset = "Pan_Numbers",keep= 'first')
Unique_records = pan_number.count()
Unique_records

Pan_Numbers    9025
dtype: int64

In [8]:
print('Total_Records',Total_Records)
print('non_missing',non_missing_pan_number)
print('Unique_records',Unique_records)


Total_Records 10000
non_missing Pan_Numbers    9033
dtype: int64
Unique_records Pan_Numbers    9025
dtype: int64


In [9]:
print(Unique_records)

Pan_Numbers    9025
dtype: int64


In [9]:
# Variable 
def has_adjacent_repition(pan):
    for i in range(len(pan)-1):
        if pan[i] == pan[i+1]:
            return True
        return False
    

In [10]:
has_adjacent_repition('AABCD')
has_adjacent_repition('ABCDE')
has_adjacent_repition('ABCDE')

True

False

False

In [11]:
def has_adjacent_repition(pan):
    return any (pan[i] == pan[i+1] for i in range(len(pan)-1))

In [12]:
has_adjacent_repition('AABCD')
has_adjacent_repition('ABCDE')

True

False

In [18]:
def is_sequectial(pan): #ABCD
    for i in range(len(pan)-1):
        if ord(pan[i+1]) - ord(pan[i]) !=1:
            return False
    return True 
        

In [20]:
is_sequectial('ABCDE')
is_sequectial('ABCEF')

True

False

In [28]:
def is_sequectial(pan):
    return all(ord(pan[i+1]) -ord(pan[i])== 1 for i in range(len(pan)-1))  #matlab agar poore string me har jagah condition satisfy hui to True, agar ek bhi break hua to False.

In [29]:
is_sequectial('ABCDE')
is_sequectial('ABCEF')

True

False

In [33]:
def is_valid_pan(pan):
    if len(pan) != 10:
        return False
        
    if not re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$', pan):  # yaha 0-5 ki jagah 0-9 hona chahiye
        return False
        
    if has_adjacent_repition(pan):   # agar adjacent repeat mila
        return False                 # toh PAN galat hai

    if is_sequectial(pan):           # agar pura sequential mila
        return False                 # toh PAN galat hai

    return True                      # baki sab case me PAN sahi hai


In [38]:
pan_number["Status"] = pan_number["Pan_Numbers"].apply(lambda x:"Valid" if is_valid_pan(x) else "Invalid")

In [41]:
valid_cnt = (pan_number["Status"] == "Valid").sum()
invalid_cnt = (pan_number["Status"] == "Invalid").sum()
missing_cnt = Total_Records - (valid_cnt + invalid_cnt)

In [44]:
print("Total_Records:",Total_Records)
print("Valid:",valid_cnt)
print("Invalid:",invalid_cnt)
print("Missing:",missing_cnt)

Total_Records: 10000
Valid: 3193
Invalid: 5832
Missing: 975


In [50]:
pan_summary = pd.DataFrame({"Total Processed Records":[Total_Records],
                            "Total Valid Records":[valid_cnt],
                            "Total Invalid Records":[invalid_cnt],
                            "Total Missing Records":[missing_cnt]})

In [51]:
pan_summary

,Total Processed Records,Total Valid Records,Total Invalid Records,Total Missing Records
0,10000,3193,5832,975


In [53]:
with pd.ExcelWriter("PAN VALIDATION RESULT By Sagar Sharma.xlsx") as writer:
    pan_number.to_excel(writer,sheet_name = "PAN Validations",index=False)
    pan_summary.to_excel(writer,sheet_name = "Summary",index=False)